## 生产环境测试
先查询一下本地的模型列表

In [1]:
!curl http://127.0.0.1:11434

Ollama is running


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    17  100    17    0     0   9513      0 --:--:-- --:--:-- --:--:-- 17000


In [1]:
!curl http://localhost:11434/api/tags

{"models":[{"name":"qwen2.5:latest","model":"qwen2.5:latest","modified_at":"2025-03-28T22:15:41.7748358+08:00","size":4683087332,"digest":"845dbda0ea48ed749caafd9e6037047aa19acfcfd82e704d7ca97d631a0b697e","details":{"parent_model":"","format":"gguf","family":"qwen2","families":["qwen2"],"parameter_size":"7.6B","quantization_level":"Q4_K_M"}}]}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   345  100   345    0     0   1671      0 --:--:-- --:--:-- --:--:--  1674


### 解析后的输出内容

以下是解析后的 JSON 数据，每行后面添加了中文注释：

```json
{
  "models": [ // 模型列表
    {
      "name": "qwen2.5:latest", // 模型名称
      "model": "qwen2.5:latest", // 模型标识符
      "modified_at": "2025-03-28T22:15:41.7748358+08:00", // 修改时间
      "size": 4683087332, // 模型文件大小（字节）
      "digest": "845dbda0ea48ed749caafd9e6037047aa19acfcfd82e704d7ca97d631a0b697e", // 模型文件的哈希值
      "details": { // 模型详细信息
        "parent_model": "", // 父模型（无）
        "format": "gguf", // 模型格式
        "family": "qwen2", // 模型系列
        "families": ["qwen2"], // 模型所属系列列表
        "parameter_size": "7.6B", // 参数规模（76亿参数）
        "quantization_level": "Q4_K_M" // 量化级别
      }
    }
  ]
}
```
---

### 将 `curl` 输出的统计部分整理成 Markdown 表格

以下是将 `curl` 输出的统计部分整理成的 Markdown 表格，并添加中文翻译：

| 字段                     | 值        | 中文说明                           |
|--------------------------|-----------|------------------------------------|
| **Total** (总进度)       | 100%      | 总体传输进度                       |
| **Received** (已接收)    | 100%      | 已接收到的数据百分比               |
| **Xferd** (已传输)       | 100%      | 已传输的数据百分比                 |
| **Average Speed** (平均速度) | 1671 B/s | 数据传输的平均速度                 |
| **Time Total** (总时间)  | --:--:--  | 总耗时（格式化为时:分:秒）         |
| **Time Spent** (已用时间) | --:--:--  | 已用时间（格式化为时:分:秒）       |
| **Time Left** (剩余时间) | --:--:--  | 预计剩余时间（格式化为时:分:秒）   |
| **Current Speed** (当前速度) | 1674 B/s | 当前瞬时传输速度                   |

---

### 具体数值说明

| 字段                     | 数值      | 中文说明                           |
|--------------------------|-----------|------------------------------------|
| **Dload** (下载量)       | 345 字节  | 下载的总数据量                     |
| **Upload** (上传量)      | 0 字节    | 上传的总数据量                     |
| **Total** (总计)         | 345 字节  | 总传输数据量                       |
| **Spent** (已用时间)     | --:--:--  | 实际传输所花费的时间               |
| **Left** (剩余时间)      | --:--:--  | 预计剩余完成时间                   |
| **Speed** (当前速度)     | 1674 B/s  | 当前瞬时传输速度                   |

---


## api调用测试

### 默认为流式输出

In [6]:
import requests
import json

response = requests.post(
    "http://localhost:11434/api/generate",
    json={"model": "qwen2.5:latest", "prompt": "天空为什么是蓝色的?"}
)
print(response.text)

{"model":"qwen2.5:latest","created_at":"2025-03-28T16:28:27.1205108Z","response":"天空","done":false}
{"model":"qwen2.5:latest","created_at":"2025-03-28T16:28:27.1832899Z","response":"之所以","done":false}
{"model":"qwen2.5:latest","created_at":"2025-03-28T16:28:27.2316655Z","response":"呈现","done":false}
{"model":"qwen2.5:latest","created_at":"2025-03-28T16:28:27.2869821Z","response":"蓝色","done":false}
{"model":"qwen2.5:latest","created_at":"2025-03-28T16:28:27.3291883Z","response":"，","done":false}
{"model":"qwen2.5:latest","created_at":"2025-03-28T16:28:27.3817908Z","response":"主要是","done":false}
{"model":"qwen2.5:latest","created_at":"2025-03-28T16:28:27.4318687Z","response":"因为","done":false}
{"model":"qwen2.5:latest","created_at":"2025-03-28T16:28:27.4866781Z","response":"大气","done":false}
{"model":"qwen2.5:latest","created_at":"2025-03-28T16:28:27.5431092Z","response":"中的","done":false}
{"model":"qwen2.5:latest","created_at":"2025-03-28T16:28:27.5865793Z","response":"气体","done":false}

### 原生流式输出展示

In [1]:
import requests
import json
from IPython.display import display, clear_output
import time

def generate_stream(model, prompt, api_url):

    # 创建请求
    response = requests.post(
        api_url,
        json={"model": model, "prompt": prompt, "stream": True},
        stream=True
    )

    # 流式输出
    full_response = ""
    for line in response.iter_lines():
        if line:
            data = json.loads(line.decode('utf-8'))
            if 'response' in data:
                chunk = data['response']
                full_response += chunk
                clear_output(wait=True)
                display(full_response)

if __name__ == "__main__":
    # 可变参数
    MODEL = "qwen2.5:latest"
    PROMPT = "天空为什么是蓝色的?"
    API_URL = "http://localhost:11434/api/generate"

    # 调用函数
    generate_stream(MODEL, PROMPT, API_URL)

'天空呈现蓝色的原因主要是因为大气散射，这种现象被称为瑞利散射（Rayleigh scattering），由英国物理学家瑞利首先提出。当太阳光进入地球的大气层时，光线会与大气中的分子和小颗粒相互作用。\n\n太阳发出的白光实际上是包含多种颜色的光谱，其中不同波长的光的颜色也不同：红色、橙色、黄色、绿色、蓝色以及紫色等。在这些不同颜色中，蓝光和紫光的波长较短，能量较高。当光线遇到大气中的分子（如氮气和氧气）时，它们会更容易被散射。\n\n由于蓝光和紫光的波长较短，因此它们在大气中会被更强烈地散射。而且人眼对蓝色比较敏感，所以天空整体呈现出蓝色。在日出和日落时分，太阳位于地平线附近，光线需要穿过更多的大气层才能到达我们的眼睛，此时大部分紫光被散射掉了，剩下的蓝光和其他颜色的光混合在一起，所以我们会看到橙色、红色或者紫色的天空。\n\n因此，天空之所以呈现蓝色，主要是因为大气对短波长（如蓝光和紫光）的光线进行强烈的散射。'

### 根据业务需求可关闭流式输出

In [2]:
import requests
import json
from IPython.display import display, clear_output
import time

def generate_non_stream(model, prompt, api_url):
    # 创建请求
    response = requests.post(
        api_url,
        json={"model": model, "prompt": prompt, "stream": False}
    )

    # 解析并显示结果
    if response.status_code == 200:
        data = response.json()
        if 'response' in data:
            display(data['response'])
    else:
        display(f"请求失败: {response.status_code}")
        display(response.text)

if __name__ == "__main__":
    # 可变参数
    MODEL = "qwen2.5:latest"
    PROMPT = "天空为什么是蓝色的?"
    API_URL = "http://localhost:11434/api/generate"

    # 调用函数 - 选择使用流式或非流式版本
    generate_non_stream(MODEL, PROMPT, API_URL)  # 非流式版本

'天空呈现蓝色的原因在于大气中的分子和其他小颗粒如何散射太阳光。当太阳光照射到地球的大气层时，它实际上是由不同颜色的光组成的，这些颜色对应着不同的波长（例如，红色光波较长，紫色光波较短）。\n\n光线进入地球大气层后，会遇到空气中的气体、水蒸气和尘埃等微粒。根据瑞利散射原理，当光线遇到比其波长大得多的小颗粒时会发生散射。由于蓝光的波长较短，因此更容易被这些小颗粒散射到各个方向上。相比之下，红光和其他较长波长的光则不容易发生这种类型的散射。\n\n日间天空呈现蓝色是因为我们看到的是从四面八方散射来的蓝色光线比其他颜色更多。当我们抬头看晴朗无云的天空时，几乎所有的角度看过去都看到了被散射到各个方向上的蓝光。而在太阳附近或者接近地平线时，由于光线需要穿过更厚的大气层，更多的蓝光和其他短波长的光都被散射掉了，剩下的主要是红色和橙色的光，因此此时天空看起来会呈现出黄色或橙色。\n\n总的来说，这就是为什么我们在白天看到的晴朗天空呈现蓝色的原因。'